### Fine-tuning LLama 2 with QLoRA
https://blog.ovhcloud.com/fine-tuning-llama-2-models-using-a-single-gpu-qlora-and-ai-notebooks/

In [ ]:
!pip install -r requirements_ovhcloud.txt

In [ ]:
!lsb_release -a

In [ ]:
!export CUDA_VISIBLE_DEVICES=0,1,2,3

In [ ]:
!sudo chmod 777 /root

In [1]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

In [2]:
f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'

'11GB'

In [3]:
def load_model(model_name, bnb_config):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # dispatch efficiently the model on the available ressources  
        load_in_8bit=True,
        #max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB',
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

### Download the dataset

In [4]:
import pandas as pd


PATH = "gs://kisai-data-msca310019-capstone/Text_Simplification/simplified_df_cefr_labeled.csv"
data = pd.read_csv(PATH)
data.head()

,source,target,source_level_og,target_level_og,data_source,id,source_level_cefr,target_level_cefr,model,prompt,generated_text,cefr_labels
0,Donald Trump is interested in buying Greenland...,Donald Trump is interested in buying Greenland...,2.0,1.0,BreakingNewsEnglish,TS000001588,1,1,flant5,Simplify,donald trump has said he would be interested i...,1
1,Everyone knows that children don't like eating...,Everyone knows children don't like eating gree...,2.0,1.0,BreakingNewsEnglish,TS000001750,1,1,flant5,Simplify,children dislike greens,0
2,Archaeologists have found a big cemetery under...,Archaeologists found a big cemetery under the ...,2.0,1.0,BreakingNewsEnglish,TS000001715,1,1,flant5,Simplify,egypt s government said we will need at least ...,1
3,Baby orangutans in Indonesia are going to scho...,Baby orangutans in Indonesia are going to scho...,2.0,1.0,BreakingNewsEnglish,TS000001895,2,1,flant5,Simplify,international animal rescue iar is a charity t...,1
4,New research says bald men should not worry ab...,Research says bald men should not worry or get...,2.0,1.0,BreakingNewsEnglish,TS000001756,1,1,flant5,Simplify,men get stressed about losing their hair,0


In [5]:
print("Length before dropping nan", len(data))
data.dropna(subset=['target_level_og', 'source_level_og'], inplace=True)
print("Length after dropping nan", len(data))

Length before dropping nan 3000
Length after dropping nan 2400


In [6]:
def generate_instruction_column(source_cefr, target_cefr):
    return f"Simplify the following context from CEFR Level {int(source_cefr)} to CEFR Level {int(target_cefr)}"

data["instruction"] = data.apply(lambda row: generate_instruction_column(row["source_level_og"], row["target_level_og"]), axis=1)

In [7]:
data.iloc[0]["instruction"]

'Simplify the following context from CEFR Level 2 to CEFR Level 1'

In [8]:
data.rename({'source': 'context', 'target': 'response'}, axis=1, inplace=True)
data = data[["instruction", "context", "response"]]

In [9]:
# Convert the DataFrame to a dictionary
data_dict = data.to_dict('list')

# Convert the dictionary to a Hugging Face Dataset
dataset = Dataset.from_dict(data_dict)

In [10]:
dataset

Dataset({
    features: ['instruction', 'context', 'response'],
    num_rows: 2400
})

In [11]:
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Number of prompts: 2400
Column names are: ['instruction', 'context', 'response']


### Preprocess the dataset

In [12]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction', 'context', 'response')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """

    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"
    
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['context']}" if sample["context"] else None
    response = f"{RESPONSE_KEY}\n{sample['response']}"
    end = f"{END_KEY}"
    
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    
    sample["text"] = formatted_prompt

    return sample

In [13]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)
    
    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, 
                                      tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "context", "response", "text"],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

### Create a bitsandbytes configuration

In [14]:
def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

In [15]:
def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        r=16,  # dimension of the updated matrices
        lora_alpha=64,  # parameter for scaling
        target_modules=modules,
        lora_dropout=0.1,  # dropout probability for layers
        bias="none",
        task_type="CAUSAL_LM",
    )

    return config

In [16]:
# SOURCE https://github.com/artidoro/qlora/blob/main/qlora.py

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [17]:
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )


### Train

In [18]:
model_name = "meta-llama/Llama-2-7b-hf" 
seed = 42

In [19]:
# Load model from HF with user's token and with bitsandbytes config

bnb_config = create_bnb_config()

model, tokenizer = load_model(model_name, bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [20]:
## Preprocess dataset

max_length = get_max_length(model)

dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [21]:
dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 2400
})

In [22]:
def train(model, tokenizer, dataset, output_dir):
    # Apply preprocessing to the model to prepare it by
    # 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # 2 - Using the prepare_model_for_kbit_training method from PEFT
    model = prepare_model_for_kbit_training(model)

    # Get lora module names
    modules = find_all_linear_names(model)

    # Create PEFT config for these modules and wrap the model to PEFT
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)
    
    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)
    
    # Training parameters
    trainer = Trainer(
        model=model,
        train_dataset=dataset,
        args=TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            max_steps=20,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=1,
            output_dir="outputs",
            optim="paged_adamw_8bit",
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )
    
    model.config.use_cache = False  # re-enable for inference to speed up predictions for similar inputs
    
    ### SOURCE https://github.com/artidoro/qlora/blob/main/qlora.py
    # Verifying the datatypes before training
    
    dtypes = {}
    for _, p in model.named_parameters():
        dtype = p.dtype
        if dtype not in dtypes: dtypes[dtype] = 0
        dtypes[dtype] += p.numel()
    total = 0
    for k, v in dtypes.items(): total+= v
    for k, v in dtypes.items():
        print(k, v, v/total)
     
    do_train = True
    
    # Launch training
    print("Training...")
    
    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)    
    
    ###
    
    # Saving model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok=True)
    trainer.model.save_pretrained(output_dir)
    
    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

In [23]:
output_dir = "results/kisai_llama2/final_checkpoint"
train(model, tokenizer, dataset, output_dir)

all params: 3,540,389,888 || trainable params: 39,976,960 || trainable%: 1.1291682911958425
torch.float32 302387200 0.08541070604255438
torch.uint8 3238002688 0.9145892939574456
Training...


wandb: Currently logged in as: egehanyorulmaz (keep-it-simple-ai). Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.897300
2,1.866300
3,1.699200
4,1.539900
5,1.317800
6,1.134400
7,1.090700
8,1.077100
9,1.131800
10,1.111400


***** train metrics *****
  epoch                    =       0.03
  total_flos               =  1010115GF
  train_loss               =     1.2045
  train_runtime            = 0:03:21.41
  train_samples_per_second =      0.397
  train_steps_per_second   =      0.099
{'train_runtime': 201.4107, 'train_samples_per_second': 0.397, 'train_steps_per_second': 0.099, 'total_flos': 1084603414462464.0, 'train_loss': 1.2045360773801803, 'epoch': 0.03}
Saving last checkpoint of the model...


### Model save

In [45]:
output_dir

'results/kisai_llama2/final_checkpoint'

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
# model = model.merge_and_unload()

output_merged_dir = "results/llama2/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok=True)
model.save_pretrained(output_merged_dir, safe_serialization=True)

# save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(output_merged_dir)

## Inference

In [ ]:
print(dataset["input_ids",])

In [48]:
torch.set_grad_enabled(False)

In [49]:
def make_inference(instruction, context = None):
    if context:
        prompt = f"Below is an instruction that describes a task, paired with an input that provides further context.\n\n### Instruction: \n{instruction}\n\n### Input: \n{context}\n\n### Response: \n"
    else:
        prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: \n{instruction}\n\n### Response: \n"
    
    inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to("cuda:0")
    outputs = model.generate(**inputs, max_new_tokens=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
    # New code to trim output
    response = response.split('End of context:')[0]
    
    filtered_response = response.split("End of response:")[0].split("Response:")[1]
    
    # display(Markdown(response))
    return filtered_response

In [36]:
from IPython.display import display, Markdown

answer = make_inference(instruction="Summarize the following context. Your answer must be in English. End the response with the phrase 'End of response.'",  context="Today Sunday 3 September (20.00) Serbia-Turkey-Euro 2023 final of women’s volleyball. TO Brussels (Belgium) the continental title is up for grabs and it will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, exciting and intense challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.")
print(answer)

Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction: 
Summarize the following context. Your answer must be in English. End the response with the phrase 'End of response.'

### Input: 
Today Sunday 3 September (20.00) Serbia-Turkey-Euro 2023 final of women’s volleyball. TO Brussels (Belgium) the continental title is up for grabs and it will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, exciting and intense challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.

### Response: 
Tonight the European women’s volleyball championship final between Serbia and Turkey. It will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, ex

Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction: 
Summarize the following context. Your answer must be in English. End the response with the phrase 'End of response.'

### Input: 
Today Sunday 3 September (20.00) Serbia-Turkey-Euro 2023 final of women’s volleyball. TO Brussels (Belgium) the continental title is up for grabs and it will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, exciting and intense challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.

### Response: 
Tonight the European women’s volleyball championship final between Serbia and Turkey. It will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, exciting and intense challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.

### End of response: 
End of response.

### 

 
Tonight the European women’s volleyball championship final between Serbia and Turkey. It will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, exciting and intense challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.

### 


In [37]:
print("a")

a


In [26]:
from IPython.display import display, Markdown

make_inference(instruction="Simplify the following context from CEFR level 3 to CEFR level 2. End the response with the phrase 'End of response.'",  context="Today Sunday 3 September (20.00) Serbia-Turkey-Euro 2023 final of women’s volleyball. TO Brussels (Belgium) the continental title is up for grabs and it will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, exciting and intense challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.")

Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction: 
Simplify the following context from CEFR level 3 to CEFR level 2. End the response with the phrase 'End of response.'

### Input: 
Today Sunday 3 September (20.00) Serbia-Turkey-Euro 2023 final of women’s volleyball. TO Brussels (Belgium) the continental title is up for grabs and it will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, exciting and intense challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.

### Response: 
Today, the final of women’s volleyball is on. It is a match between Serbia and Turkey. Serbia beat Italy to get to the final. It will be a good match.

### End of response: 

### 

In [27]:
from IPython.display import display, Markdown

make_inference(instruction="Simplify the following context from CEFR level 2 to CEFR level 1. End the response with the phrase 'End of response.'",  context="Today Sunday 3 September (20.00) Serbia-Turkey-Euro 2023 final of women’s volleyball. TO Brussels (Belgium) the continental title is up for grabs and it will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, exciting and intense challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.")

Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction: 
Simplify the following context from CEFR level 2 to CEFR level 1. End the response with the phrase 'End of response.'

### Input: 
Today Sunday 3 September (20.00) Serbia-Turkey-Euro 2023 final of women’s volleyball. TO Brussels (Belgium) the continental title is up for grabs and it will be decided who will succeed Italy in the golden register. It promises to be a particularly balanced and compelling, exciting and intense challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.

### Response: 
Today Sunday 3 September (20.00) Serbia-Turkey-Euro 2023 final of women’s volleyball. The continental title is up for grabs. It promises to be a particularly balanced and compelling challenge, open to any result. It will be a cross between Italian coaches, given that coach Daniele Santarelli sits on the Anatolian bench, capable of beating Italy in the tie-break, and Giovanni Guidetti is on the Balkan bench.

### End of response: 
End of response.

### 

In [28]:
from IPython.display import display, Markdown

# ANSWER: Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.

make_inference(instruction="When did Virgin Australia start operating?",  
               context="Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.")

Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction: 
When did Virgin Australia start operating?

### Input: 
Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.

### Response: 
Virgin Australia started operating on August 31st, 2000.

### 

In [29]:
from IPython.display import display, Markdown

# ANSWER: No. Stalemate is a drawn position. It doesn't matter who has captured more pieces or is in a winning position

make_inference(instruction="If I have more pieces at the time of stalemate, have I won?",  
               context="Stalemate is a situation in chess where the player whose turn it is to move is not in check and has no legal move. Stalemate results in a draw. During the endgame, stalemate is a resource that can enable the player with the inferior position to draw the game rather than lose. In more complex positions, stalemate is much rarer, usually taking the form of a swindle that succeeds only if the superior side is inattentive.[citation needed] Stalemate is also a common theme in endgame studies and other chess problems. The outcome of a stalemate was standardized as a draw in the 19th century. Before this standardization, its treatment varied widely, including being deemed a win for the stalemating player, a half-win for that player, or a loss for that player; not being permitted; and resulting in the stalemated player missing a turn. Stalemate rules vary in other games of the chess family.")

Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction: 
If I have more pieces at the time of stalemate, have I won?

### Input: 
Stalemate is a situation in chess where the player whose turn it is to move is not in check and has no legal move. Stalemate results in a draw. During the endgame, stalemate is a resource that can enable the player with the inferior position to draw the game rather than lose. In more complex positions, stalemate is much rarer, usually taking the form of a swindle that succeeds only if the superior side is inattentive.[citation needed] Stalemate is also a common theme in endgame studies and other chess problems. The outcome of a stalemate was standardized as a draw in the 19th century. Before this standardization, its treatment varied widely, including being deemed a win for the stalemating player, a half-win for that player, or a loss for that player; not being permitted; and resulting in the stalemated player missing a turn. Stalemate rules vary in other games of the chess family.

### Response: 
No, I have not won.

### Endpoint: 
Stalemate is a draw.

### Context: 
Stalemate is a situation in chess where the player whose turn it is to move is not in check and has no legal move. Stalemate results in a draw. During the endgame, stalemate is a resource that can enable the player with the inferior position to draw the game rather than lose. In more complex positions, stalemate is much rarer, usually taking the form of a swindle that succeeds only if the superior side is inattentive. Stalemate is also a common theme in endgame studies and other chess problems. The outcome of a stalemate was standardized as a draw in the 19th century. Before this standardization, its treatment varied widely, including being deemed a win for the stalemating player, a half-win for that player, or a loss for that player; not being permitted; and resulting in the stalemated player missing a turn. Stalemate rules vary in other games of the chess family.

### Endpoint: 
Stalem

In [30]:
from IPython.display import display, Markdown

# ANSWER: The top speed of a Kia Stinger is 269km/h (167mph) according to this text.

make_inference(instruction="Given this paragraph, what is the top speed of a Kia Stinger?",  
               context="Kia claims that the Stinger accelerates from zero to 100 km/h (62 mph) in 7.7, 6 and 4.9 seconds for the 2.2-liter diesel, 2.0-liter petrol and 3.3-liter petrol respectively. Schreyer reportedly drove a pre-production Stinger GT at a top speed of 269 km/h (167 mph) on the Autobahn. During a test by Car and Driver, an all-wheel-drive U.S. spec GT 3.3T with Michelin Pilot Sport 4 tires achieved 0–60 mph (0–97 km/h) in 4.6 seconds on the track, reached 0.91 g on the skidpad and was able to stop from 70 mph (113 km/h) in 164 feet (50 m). According to this publication, the U.S. model's top speed is governed at 167 mph (269 km/h) per Kia specs. In tests conducted by Motor Trend, the four-cylinder U.S. spec Stinger 2.0 RWD on Bridgestone Potenza tires reached 60 mph (97 km/h) in 6.6 seconds, completed the 1⁄4-mile (0.4 km) run in 15 seconds and stopped from 60 mph (97 km/h) in 126 feet (38 m). The average lateral acceleration recorded in track testing was 0.85 g.")

Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction: 
Given this paragraph, what is the top speed of a Kia Stinger?

### Input: 
Kia claims that the Stinger accelerates from zero to 100 km/h (62 mph) in 7.7, 6 and 4.9 seconds for the 2.2-liter diesel, 2.0-liter petrol and 3.3-liter petrol respectively. Schreyer reportedly drove a pre-production Stinger GT at a top speed of 269 km/h (167 mph) on the Autobahn. During a test by Car and Driver, an all-wheel-drive U.S. spec GT 3.3T with Michelin Pilot Sport 4 tires achieved 0–60 mph (0–97 km/h) in 4.6 seconds on the track, reached 0.91 g on the skidpad and was able to stop from 70 mph (113 km/h) in 164 feet (50 m). According to this publication, the U.S. model's top speed is governed at 167 mph (269 km/h) per Kia specs. In tests conducted by Motor Trend, the four-cylinder U.S. spec Stinger 2.0 RWD on Bridgestone Potenza tires reached 60 mph (97 km/h) in 6.6 seconds, completed the 1⁄4-mile (0.4 km) run in 15 seconds and stopped from 60 mph (97 km/h) in 126 feet (38 m). The average lateral acceleration recorded in track testing was 0.85 g.

### Response: 
The top speed of a Kia Stinger is 274 km/h (170 mph).

### End:

### End of Context:

### End of Response:

### Response: 
The top speed of a Kia Stinger is 274 km/h (170 mph).

### End of Context:

### End of Response:

### Feedback:

Good job!

### End of Feedback:

### Begin Extra Response:

### Extra Response:

### End of Extra Response:

### End of Context:

### End of Response:

### Feedback:

Very good job!

### End of Feedback:

### Begin Extra Response:

### Extra Response:

### End of Extra Response:

### End of Context:

### End of Response:

### Feedback:

Great job!

### End of Feed

In [38]:
data.head()

,instruction,context,response
0,Simplify the following context from CEFR Level...,Donald Trump is interested in buying Greenland...,Donald Trump is interested in buying Greenland...
1,Simplify the following context from CEFR Level...,Everyone knows that children don't like eating...,Everyone knows children don't like eating gree...
2,Simplify the following context from CEFR Level...,Archaeologists have found a big cemetery under...,Archaeologists found a big cemetery under the ...
3,Simplify the following context from CEFR Level...,Baby orangutans in Indonesia are going to scho...,Baby orangutans in Indonesia are going to scho...
4,Simplify the following context from CEFR Level...,New research says bald men should not worry ab...,Research says bald men should not worry or get...


In [39]:
data.iloc[0]["context"]

'Donald Trump is interested in buying Greenland. He said it would be like buying property and was, "a large real estate deal". He added: "They\'ve got a lot of valuable minerals." Mr Trump said: "Denmark owns it....We protect Denmark....So the concept came up and I said, \'Certainly I\'d be [interested].\' He said buying Greenland was not his top priority at the moment. He said: "It\'s not number one on the burner."'

In [40]:
data.iloc[0]["response"]

'Donald Trump is interested in buying Greenland. He said it would be, "a large real estate deal". He added: "They\'ve got a lot of valuable minerals." Mr Trump said: "Certainly I\'d be [interested].\' He said buying Greenland was not his top priority. He said: "It\'s not number one on the burner."'

In [41]:
data["model_response"] = ""

In [42]:
PATH = "gs://kisai-data-msca310019-capstone/Text_Simplification/finetuned_llama2_responses.csv"

In [ ]:
import time

for idx, row in data.iterrows():
    print("Currently processing index ", idx)
    start = time.perf_counter()
    answer = make_inference(instruction=row["instruction"],  
                            context=row["context"])
    data.at[idx, 'model_response'] = answer
    end = time.perf_counter()

    print(f"Finished processing index {idx} in {end-start} seconds.")
    
    if idx % 20 == 0:
        print(f"Saving for batch {idx//20}")
        start = time.perf_counter()
        data.to_csv(PATH, index=False)
        end = time.perf_counter()
        print(f"Finishes saving the batch {idx//20} in {end-start} seconds")

Currently processing index  0
Finished processing index 0 in 228.16458519599837 seconds.
Saving for batch 0
Finishes saving the batch 0 in 0.25415011800214415 seconds
Currently processing index  1
Finished processing index 1 in 216.31794119100232 seconds.
Currently processing index  2


In [ ]:
print("a")